<a href="https://colab.research.google.com/github/ciaoCiao100/mushroom_kaggle/blob/master/mushroom_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# vengono importate le librerie utili 

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# operazione che fa in modo che i numeri siano più leggibili
np.set_printoptions(precision=3, suppress=True)

# Download del dataset e lettura generica 

In [46]:
data_path = "/content/drive/My Drive/mushroom_data_all.csv"
df = pd.read_csv(data_path)
target_label ="class_edible"
df.head()

,class_edible,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


# conversione degli elementi presenti in tabella in interi

In [47]:
#tutte le colonne che contengono stringhe devono essere trasformate in colonne che contengono interi 
for elemento in df:
 
  df[elemento] = pd.Categorical(df[elemento])
  df[elemento] = getattr(df, elemento).cat.codes
# mostro il dataset dopo l'operazione
df.head()



,class_edible,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1


# preparazione del modello di rete neurale 

In [0]:
#costruisco il modello
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

model = get_compiled_model()
#preparo il dataset
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

# addestramento

In [49]:
# Creazione  del training
train_target = train.pop(target_label)
train_dataset = (tf.data.Dataset.from_tensor_slices((train.values, train_target.values))).shuffle(len(train)).batch(1)
#addestramento e impostazione epoche
model.fit(train_dataset, epochs=20)

Epoch 1/20
5199/5199 [==============================] - 6s 1ms/step - loss: 0.3240 - accuracy: 0.8757
Epoch 2/20
5199/5199 [==============================] - 6s 1ms/step - loss: 0.1137 - accuracy: 0.9638
Epoch 3/20
5199/5199 [==============================] - 6s 1ms/step - loss: 0.0639 - accuracy: 0.9777
Epoch 4/20
5199/5199 [==============================] - 6s 1ms/step - loss: 0.0472 - accuracy: 0.9854
Epoch 5/20
5199/5199 [==============================] - 6s 1ms/step - loss: 0.0281 - accuracy: 0.9900
Epoch 6/20
5199/5199 [==============================] - 6s 1ms/step - loss: 0.0221 - accuracy: 0.9917
Epoch 7/20
5199/5199 [==============================] - 6s 1ms/step - loss: 0.0179 - accuracy: 0.9933
Epoch 8/20
5199/5199 [==============================] - 6s 1ms/step - loss: 0.0112 - accuracy: 0.9958
Epoch 9/20
5199/5199 [==============================] - 6s 1ms/step - loss: 0.0090 - accuracy: 0.9965
Epoch 10/20
5199/5199 [==============================] - 6s 1ms/step - loss: 0.011

## test

In [50]:
# Creazione del  test 
test_target = test.pop(target_label)
test_dataset = (tf.data.Dataset.from_tensor_slices((test.values, test_target.values))).shuffle(len(test)).batch(1)

# Fase di test
test_loss, test_accuracy = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

1625/1625 [==============================] - 1s 862us/step - loss: 0.0814 - accuracy: 0.9742


Test Loss 0.08140578866004944, Test Accuracy 0.9741538166999817


# valutazione

In [51]:
#valutazione
val_target = val.pop(target_label)

val_dataset = (tf.data.Dataset.from_tensor_slices(val.values)).batch(1)

# Fase di valutazione
predictions = model.predict(val_dataset)


# Show some results 
for prediction, avvelenato in zip(predictions[:10], list(val_target)[:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted avvelenato: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("AVVELENATO" if bool(avvelenato) else "SICURO"))


Predicted avvelenato: 100.00%  | Actual outcome:  AVVELENATO
Predicted avvelenato: 0.00%  | Actual outcome:  SICURO
Predicted avvelenato: 100.00%  | Actual outcome:  AVVELENATO
Predicted avvelenato: 0.00%  | Actual outcome:  SICURO
Predicted avvelenato: 0.00%  | Actual outcome:  SICURO
Predicted avvelenato: 100.00%  | Actual outcome:  AVVELENATO
Predicted avvelenato: 100.00%  | Actual outcome:  AVVELENATO
Predicted avvelenato: 0.00%  | Actual outcome:  SICURO
Predicted avvelenato: 100.00%  | Actual outcome:  AVVELENATO
Predicted avvelenato: 100.00%  | Actual outcome:  AVVELENATO
